In [1]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://w11bw9q2bp-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [2]:
#import random 
import numpy as np 
import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub


# step 1: import Flask 
from flask import Flask, request

# step 2: define Flask app 
# app = Flask("My Flask App")
app = Flask(__name__)


# step 3: define pages 

@app.route('/')
def home():
  return '<h1> Hello from Flask </h1>'

@app.route('/help')
def help():
  return '<h1> Hello from help page </h1>'


@app.route('/add')
def add():
  # to request page 
  # /add?x=4&y=4
  x = int(request.args.get('x'))
  y = int(request.args.get('y'))
  result = x + y 
  return '<h1> {}+{}={} </h1>'.format(x, y, result)


##################################
classifier_model ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4" 
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_model, input_shape=IMAGE_SHAPE+(3,))
])

labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

@app.route('/img')
def img():
  url = request.args.get('url')
  # get image and preprocess (open, resize, convert to gray scale)
  grace_hopper = tf.keras.utils.get_file('./image.jpg', url)
  grace_hopper = Image.open(grace_hopper).resize(IMAGE_SHAPE)
  grace_hopper = np.array(grace_hopper)/255.0
  # classify 
  result = classifier.predict(grace_hopper[np.newaxis, ...])
  predicted_class = np.argmax(result[0], axis=-1)
  predicted_class_name = imagenet_labels[predicted_class]
  return '<h1> prediction: {} </h1>'.format(predicted_class_name)



# step 4: run app 
if __name__ == "__main__":  
  app.run()






16384/10484 [==============================================] - 0s 0us/step
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [02/Nov/2020 19:29:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2020 19:29:56] "GET /favicon.ico HTTP/1.1" 404 -


155648/152265 [==============================] - 0s 1us/step


INFO:werkzeug:127.0.0.1 - - [02/Nov/2020 19:30:10] "GET /img?url=https%3A%2F%2Fcdn.shopify.com%2Fs%2Ffiles%2F1%2F0035%2F2754%2F0782%2Farticles%2FInternational_Flower_Day_1200x1200.jpeg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Nov/2020 19:30:10] "GET /favicon.ico HTTP/1.1" 404 -
